# A Machine Learning Framework for Stroke Prediction: Balancing Precision and Recall in Healthcare Analytics (Master Notebook)

In [ ]:
import papermill
import scrapbook
from itertools import product
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import joblib
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import seaborn as sns
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import math



In [ ]:
dataset = pd.read_csv('healthcare-dataset-stroke-data.csv')
dataset

## Data Visualization

In [ ]:
sns.set(style='whitegrid', context='talk')

stroke_data = dataset[dataset['stroke'] == 1].drop('id', axis=1).describe().T
no_stroke_data = dataset[dataset['stroke'] == 0].drop('id', axis=1).describe().T

cmap = sns.diverging_palette(220, 20, as_cmap=True)

fig, axes = plt.subplots(1, 2, figsize=(14, 7))

sns.heatmap(stroke_data[['mean']], annot=True, cmap=cmap, linewidths=0.8,
            linecolor='gray', cbar=True, fmt='.2f', ax=axes[0],
            annot_kws={"size":14})
axes[0].set_title('Stroke Suffered', fontsize=16, pad=16)
axes[0].set_ylabel('')
axes[0].set_xlabel('')
axes[0].tick_params(labelsize=12)

sns.heatmap(no_stroke_data[['mean']], annot=True, cmap=cmap, linewidths=0.8,
            linecolor='gray', cbar=True, fmt='.2f', ax=axes[1],
            annot_kws={"size":14})
axes[1].set_title('No Stroke Suffered', fontsize=16, pad=16)
axes[1].set_ylabel('')
axes[1].set_xlabel('')
axes[1].tick_params(labelsize=12)

fig.suptitle('Mean Statistics Comparison for Stroke Data', fontsize=18, y=1.05)
plt.tight_layout()
plt.show()

### From this insight, we can notice that people that are more likely to suffer from a stroke are the ones that are elder and have a high averge glucose level, indicating diabetes. Moreover, heart disease and hypertension can contribute to being affected by a stroke in the near future. Thus, BMI doesn't seem as relevant as the other features.

In [ ]:
counts = dataset['stroke'].value_counts()
labels = ['No Stroke Suffered', 'Stroke Suffered']
percentages = [counts[0] / counts.sum() * 100, counts[1] / counts.sum() * 100]

colors = ['#66b3ff', '#ff9999']
explode = (0.1, 0)

fig, ax = plt.subplots(figsize=(10, 7))

wedges, texts, autotexts = ax.pie(
    percentages,
    labels=labels,
    autopct='%1.1f%%',
    startangle=90,
    explode=explode,
    colors=colors,
    shadow=True,
    wedgeprops={'edgecolor': 'black', 'linewidth': 1}
)

ax.set_title('Stroke Events (%)', fontsize=16)
plt.tight_layout()
plt.show()

### We are dealing with a highly imbalanced dataset, where only 4.9% of the samples are from people that have suffered a stroke.

## Data Cleaning

In [ ]:
print(dataset.isnull().sum())

### We have 201 nulls in the BMI column that we have to deal with. We have several options: remove the affected rows, remove the affected columns, replace with the mean, or train a model to predict the missing BMI. Let's take a look at the mean of stroke and non-stroke data.

In [ ]:
dataset['bmi'] = dataset['bmi'].fillna(dataset['bmi'].mean())

In [ ]:
print(dataset.duplicated().sum())

### There are no duplicated rows in the dataset.

In [ ]:
dataset

### The ID column should also be removed as it doesn't pose relevancy to the training process.

In [ ]:
dataset.drop('id', axis=1, inplace=True)

In [ ]:
dataset

In [ ]:
categorical_features = [
    col for col in dataset.columns 
    if dataset[col].dtype == 'object' or dataset[col].nunique() <= 10  
]


numerical_features = [col for col in dataset.columns if col not in categorical_features]

categorical_features = [dataset.columns.get_loc(col) for col in categorical_features]
numerical_features = [dataset.columns.get_loc(col) for col in numerical_features]
categorical_features, numerical_features

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import math

categorical_cols = [
    col for col in dataset.columns
    if dataset[col].dtype == 'object' or dataset[col].nunique() <= 10
]

plots_per_row = 3
rows = math.ceil(len(categorical_cols) / plots_per_row)

fig, axes = plt.subplots(rows, plots_per_row, figsize=(5 * plots_per_row, 4 * rows))

axes = axes if isinstance(axes, np.ndarray) else [[axes]]

for i, col in enumerate(categorical_cols):
    r = i // plots_per_row  
    c = i % plots_per_row   
    sns.countplot(x=col, data=dataset, palette='viridis', ax=axes[r][c])
    axes[r][c].set_title(f"Distribution of {col}")
    axes[r][c].tick_params(axis='x', rotation=45)

total_subplots = rows * plots_per_row
for j in range(i+1, total_subplots):
    r = j // plots_per_row
    c = j % plots_per_row
    axes[r][c].set_visible(False)

fig.tight_layout()
plt.show()

### From here, we can think of some feature engineering steps. For instance, there is one individual with 'Other' gender category. Since there isn't a lot of data for the 'Other' category, it will pose extra challange for a model. Thus, the 'Other' individual has been removed.

In [ ]:
dataset = dataset[dataset['gender'] != 'Other']

In [ ]:
numerical_cols = [
    col for col in dataset.columns
    if np.issubdtype(dataset[col].dtype, np.number) and dataset[col].nunique() > 10
]

plots_per_row = 3
rows = math.ceil(len(numerical_cols) / plots_per_row)

fig, axes = plt.subplots(rows, plots_per_row, figsize=(5 * plots_per_row, 4 * rows))
axes = np.atleast_2d(axes)  

for i, col in enumerate(numerical_cols):
    r = i // plots_per_row
    c = i % plots_per_row
    sns.histplot(data=dataset, x=col, kde=True, ax=axes[r, c], color='skyblue')
    axes[r, c].set_title(f"Distribution of {col}")
    axes[r, c].set_xlabel(col)
    
total_subplots = rows * plots_per_row
for j in range(i + 1, total_subplots):
    r = j // plots_per_row
    c = j % plots_per_row
    axes[r, c].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
numerical_cols = [
    col for col in dataset.columns
    if np.issubdtype(dataset[col].dtype, np.number) and dataset[col].nunique() > 10
]


plots_per_row = 3
rows = math.ceil(len(numerical_cols) / plots_per_row)

fig, axes = plt.subplots(rows, plots_per_row, figsize=(5 * plots_per_row, 4 * rows))
axes = np.atleast_2d(axes)  

for i, col in enumerate(numerical_cols):
    r = i // plots_per_row
    c = i % plots_per_row
    sns.histplot(data=dataset, x=col, kde=True, ax=axes[r, c], color='skyblue')
    axes[r, c].set_title(f"Distribution of {col}")
    axes[r, c].set_xlabel(col)
    
total_subplots = rows * plots_per_row
for j in range(i + 1, total_subplots):
    r = j // plots_per_row
    c = j % plots_per_row
    axes[r, c].set_visible(False)

plt.tight_layout()
plt.show()

### Specifically, we can check the distribution relative to the two classes (stroke vs non-stroke).

In [ ]:
categorical_cols = [
    col for col in dataset.columns
    if dataset[col].dtype == 'object' or dataset[col].nunique() <= 10
]

plots_per_row = 3
rows = math.ceil(len(categorical_cols) / plots_per_row)

fig, axes = plt.subplots(rows, plots_per_row, figsize=(5 * plots_per_row, 4 * rows))
axes = axes if isinstance(axes, np.ndarray) else [[axes]]

for i, col in enumerate(categorical_cols):
    r = i // plots_per_row
    c = i % plots_per_row
    
    sns.countplot(x=col, hue='stroke', data=dataset, palette='viridis', ax=axes[r][c])
    axes[r][c].set_title(f"Distribution of {col} by Stroke Status")
    axes[r][c].tick_params(axis='x', rotation=45)

total_subplots = rows * plots_per_row
for j in range(i+1, total_subplots):
    r = j // plots_per_row
    c = j % plots_per_row
    axes[r][c].set_visible(False)

fig.tight_layout()
plt.show()

### Having the 'children' and 'never_worked' category at the same time can be tricky. That's because never worked doesn't seem to have a lot of data in general, while there are more records of children. Moreover, childrens can also be considered to be people that never worked. So, a decision can be merging the children and never_worked type for the work_type class into one: never_worked. In essence, this difference between individuals can also be made via the 'age' category.

In [ ]:
dataset[(dataset['work_type'] == 'children') & (dataset['ever_married'] == 'Yes')]

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=dataset, x='age', hue='work_type', multiple='stack', palette='viridis')
plt.title('Distribution of Work Type by Age')
plt.show()


### We can also notice this in the graph above. Therefore, a merging between 'Children' and 'Never Worked' is valid.

In [ ]:
dataset.loc[(dataset['work_type'] == 'children'), 'work_type'] = 'Never_worked'

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=dataset, x='age', hue='work_type', multiple='stack', palette='viridis')
plt.title('Distribution of Work Type by Age')
plt.show()

In [ ]:
numerical_cols = [
    col for col in dataset.columns 
    if dataset[col].dtype != 'object' and dataset[col].nunique() > 10 and 'id' not in col.lower()
]

plots_per_row = 3
rows = math.ceil(len(numerical_cols) / plots_per_row)

fig, axes = plt.subplots(rows, plots_per_row, figsize=(5 * plots_per_row, 4 * rows))
axes = np.atleast_2d(axes) 

for i, col in enumerate(numerical_cols):
    r = i // plots_per_row
    c = i % plots_per_row
    
    sns.histplot(data=dataset, x=col, hue='stroke', palette='viridis', ax=axes[r, c], kde=True)
    axes[r, c].set_title(f"Distribution of {col} by Stroke Status")
    axes[r, c].tick_params(axis='x', rotation=45)

total_subplots = rows * plots_per_row
for j in range(i + 1, total_subplots):
    r = j // plots_per_row
    c = j % plots_per_row
    axes[r, c].set_visible(False)

fig.tight_layout()
plt.show()

## Splitting the data for the workers

### Creating the train-val-test sets

In [ ]:
dataset

In [ ]:
def train_val_test_split(X, y, test_size=0.2, val_size=0.2, random_state=42):

    temp_size = test_size + val_size
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=temp_size, stratify=y, random_state=random_state
    )
    
    test_frac = test_size / temp_size
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=test_frac, stratify=y_temp, random_state=random_state
    )
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
X = dataset.drop(columns=['stroke'])
y = dataset['stroke']

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X, y, test_size=0.15, val_size=0.15)

In [ ]:
dataset_train = pd.concat([X_train, y_train], axis=1)
dataset_val = pd.concat([X_val, y_val], axis=1)
dataset_test = pd.concat([X_test, y_test], axis=1)

dataset_train.to_csv('dataset_train.csv', index=False)
dataset_val.to_csv('dataset_val.csv', index=False)
dataset_test.to_csv('dataset_test.csv', index=False)

## Papermill setup

In [ ]:
def execute_notebook(*args, parameters={}, **kwargs):
    return papermill.execute_notebook(*args, parameters=parameters, **kwargs)

In [ ]:
name = 'final_models'

models_path = 'workers/models/' + name
workers_path = 'workers/notebooks/' + name

for path in [models_path, workers_path]:
    if not os.path.exists(path):
        os.makedirs(path)

imbalanced_action = [0, 1, 2, 3, 4]
use_PCA = [False]
scale = [False, True]
normalize = [False, True]
feature_binning = [False, True]
feature_selection_mode = [0, 1, 2]
k_features = [2, 3, 4]
threshold = [0.5]
model_type = [0, 1, 2, 3, 4, 5, 6]

parameters = []

for imb in imbalanced_action:
    for pca in use_PCA:
        for sc in scale:
            for nm in normalize:
                for fe in feature_binning:
                    for fs in feature_selection_mode:
                        for k in k_features:
                            for t in threshold:
                                for mt in model_type:
                                    if fe == True:
                                        nm = False
                                        sc = False
                                    if fs == 2:
                                        k = 0
                                    bp = {
                                        'imbalanced_action': imb,
                                        'use_PCA': pca,
                                        'scale': sc,
                                        'normalize': nm,
                                        'feature_binning': fe,
                                        'feature_selection_mode': fs,
                                        'k_features': k,
                                        'threshold': t,
                                        'model_type': mt
                                    }
                                    if bp not in parameters:
                                        parameters.append(bp)

In [ ]:
len(parameters)

## Models training

In [ ]:
with ThreadPoolExecutor() as pool, tqdm(total=len(parameters)) as progress:
    for idx, param in enumerate(parameters):
        param['model_file_path'] = f'{models_path}/stroke_model_{idx}.pkl'
        param['pca_file_path'] = f'{models_path}/stroke_pca_{idx}.pkl'
        param['scaler_file_path'] = f'{models_path}/stroke_scaler_{idx}.pkl'
        param['normalizer_file_path'] = f'{models_path}/stroke_normalizer_{idx}.pkl'
        execute_notebook(
            'stroke_detection_worker_best.ipynb',
            f'{workers_path}/stroke_model_{idx}.ipynb',
            parameters=param
        )

## Extracting results from workers

In [ ]:
directory_path = workers_path
all_notebooks = {}

for filename in os.listdir(directory_path):
    if filename.endswith('.ipynb'):
        notebook_path = os.path.join(directory_path, filename)
        notebook = scrapbook.read_notebook(notebook_path)
        all_notebooks[filename] = notebook


In [ ]:
all_data = []

for filename, nb in all_notebooks.items():
    try:
        precision_train = nb.scraps['precision_train'].data
        recall_train = nb.scraps['recall_train'].data
        f1_train = nb.scraps['f1_score_train'].data
        accuracy_train = nb.scraps['accuracy_score_train'].data

        precision = nb.scraps['precision'].data
        recall = nb.scraps['recall'].data
        f1 = nb.scraps['f1_score'].data
        accuracy = nb.scraps['accuracy'].data
        pr_auc = nb.scraps['pr_auc'].data

        precision_optimal = nb.scraps['precision_optimal_f1'].data
        recall_optimal = nb.scraps['recall_optimal_f1'].data
        f1_optimal = nb.scraps['f1_score_optimal_thres'].data
        mcc_optimal = nb.scraps['mcc_optimal_f1'].data

        precision_f2 = nb.scraps['precision_optimal_f2'].data
        recall_f2 = nb.scraps['recall_optimal_f2'].data
        f2 = nb.scraps['f2_score_optimal_thres'].data
        mcc_f2 = nb.scraps['mcc_optimal_f2'].data


        params = nb.parameters

        data = params.copy()

        data['precision_train'] = precision_train
        data['recall_train'] = recall_train
        data['f1_train'] = f1_train
        data['accuracy_train'] = accuracy_train

        data['precision_val'] = precision
        data['recall_val'] = recall
        data['f1_val'] = f1
        data['accuracy_val'] = accuracy
        data['pr_auc_val'] = pr_auc

        data['precision_optimal'] = precision_optimal
        data['recall_optimal'] = recall_optimal
        data['f1_optimal'] = f1_optimal
        data['mcc_optimal'] = mcc_optimal

        data['precision_optimal_f2'] = precision_f2
        data['recall_optimal_f2'] = recall_f2
        data['f2_optimal'] = f2
        data['mcc_optimal_f2'] = mcc_f2
        

        all_data.append(data)
    except Exception as e:
        print(f'Error in {filename}: {e}')
        continue




In [ ]:
results = pd.DataFrame(all_data)
results_without_model_path = results.drop(columns=['model_file_path'])
results_without_model_path = results_without_model_path.drop(columns=['pca_file_path'])
results_without_model_path = results_without_model_path.drop(columns=['scaler_file_path'])

results_without_model_path['model_type'] = results_without_model_path['model_type'].replace(
    {0: 'Logistic Regression', 1: 'SVM', 2: 'Random Forest', 3: 'KNN', 4: 'Neural Network', 5: 'Decision Tree', 6: 'Naive Bayes'})

In [ ]:
results_without_model_path = results_without_model_path[results_without_model_path['f1_optimal'] != 'nan']
results_without_model_path = results_without_model_path[results_without_model_path['f2_optimal'] != 'nan']

results_without_model_path = results_without_model_path.drop(columns=['accuracy_train', 'precision_val', 'recall_val', 'f1_val', 'accuracy_val', 'precision_optimal', 'recall_optimal', 'f1_optimal', 'mcc_optimal'])

In [ ]:
results_without_model_path = results_without_model_path[results_without_model_path['model_type'] != 'Decision Tree']

In [ ]:
results_without_model_path = results_without_model_path.sort_values(by='f2_optimal', ascending=False)

In [ ]:
results_best = results_without_model_path[results_without_model_path['recall_optimal_f2'] > 0.71]

results_best

In [ ]:
print(results_best.to_string())

## Experiments

In [ ]:
avg_f2 = results_without_model_path.groupby(['model_type', 'feature_binning'])['f2_optimal'].mean().reset_index()

avg_f2_pivot = avg_f2.pivot(index='model_type', columns='feature_binning', values='f2_optimal')

avg_f2_pivot = avg_f2_pivot.rename(columns={False: 'No Binning', True: 'Binning'})

avg_f2_pivot.plot(kind='bar', figsize=(10, 6))
plt.title("Average F2 Score by Model Type and Feature Binning")
plt.xlabel("Model Type")
plt.ylabel("Average F2 Score")
plt.xticks(rotation=45)
plt.legend(title="Feature Binning")
plt.tight_layout()
plt.show()

In [ ]:
avg_f2 = results_without_model_path.groupby(['model_type', 'imbalanced_action'])['f2_optimal'].mean().reset_index()

avg_f2_pivot = avg_f2.pivot(index='model_type', columns='imbalanced_action', values='f2_optimal')

mapping = {0: 'None', 1: 'SMOTE', 2: 'Undersampling', 3: 'Oversampling', 4: 'Combined'}
avg_f2_pivot = avg_f2_pivot.rename(columns=mapping)

avg_f2_pivot.plot(kind='bar', figsize=(10, 6))
plt.title("Average F2 Score by Model Type and Imbalanced Action")
plt.xlabel("Model Type")
plt.ylabel("Average F2 Score")
plt.xticks(rotation=45)
plt.legend(title="Imbalanced Action")
plt.tight_layout()
plt.show()

In [ ]:

avg_f2_fs = results_without_model_path.groupby(['model_type', 'feature_selection_mode'])['f2_optimal'].mean().reset_index()

avg_f2_fs_pivot = avg_f2_fs.pivot(index='model_type', columns='feature_selection_mode', values='f2_optimal')

mapping_fs = {0: 'Chi/ANOVA', 1: 'RF Importance', 2: 'None'}
avg_f2_fs_pivot = avg_f2_fs_pivot.rename(columns=mapping_fs)

avg_f2_fs_pivot.plot(kind='bar', figsize=(10, 6))
plt.title("Average F2 Score by Model Type and Feature Selection Mode")
plt.xlabel("Model Type")
plt.ylabel("Average F2 Score")
plt.xticks(rotation=45)
plt.legend(title="Feature Selection Mode")
plt.tight_layout()
plt.show()

In [ ]:

avg_f2_scale = results_without_model_path.groupby(['model_type', 'scale'])['f2_optimal'].mean().reset_index()

avg_f2_scale_pivot = avg_f2_scale.pivot(index='model_type', columns='scale', values='f2_optimal')
avg_f2_scale_pivot = avg_f2_scale_pivot.rename(columns={True: 'Scaled', False: 'Not Scaled'})

avg_f2_norm = results_without_model_path.groupby(['model_type', 'normalize'])['f2_optimal'].mean().reset_index()


avg_f2_norm_pivot = avg_f2_norm.pivot(index='model_type', columns='normalize', values='f2_optimal')

avg_f2_norm_pivot = avg_f2_norm_pivot.rename(columns={True: 'Normalized', False: 'Not Normalized'})

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

avg_f2_scale_pivot.plot(kind='bar', ax=axes[0])
axes[0].set_title("Average F2 Score by Model Type and Scale")
axes[0].set_xlabel("Model Type")
axes[0].set_ylabel("Average F2 Score")
axes[0].tick_params(axis='x', rotation=45)
axes[0].legend(title="Scale")

avg_f2_norm_pivot.plot(kind='bar', ax=axes[1])
axes[1].set_title("Average F2 Score by Model Type and Normalization")
axes[1].set_xlabel("Model Type")
axes[1].set_ylabel("Average F2 Score")
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend(title="Normalization")

plt.tight_layout()
plt.show()